## week 3 part 1

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [6]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        neighborhoodList.append(cells[2].text.rstrip('\n'))
        boroughList.append(cells[1].text)
        postalCodeList.append(cells[0].text)

In [8]:
postalCodeList = [item.rstrip('\n') for item in postalCodeList]

In [9]:
boroughList = [item.rstrip('\n') for item in boroughList]

In [10]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [15]:
toronto_df_grouped.shape

(103, 3)

## Week 3 part 2

In [16]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.1+ KB


In [17]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## Week 3 Part 3

In [20]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5).add_to(map_toronto)  
    
map_toronto

# get certain boroughs

In [40]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_york = []

for x in borough_names:
    if "york" in x.lower():
        borough_with_york.append(x)
        
borough_with_york

['North York', 'East York', 'York']

In [41]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_york)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(34, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [43]:
CLIENT_ID = 'KTN2YDVCZLG44W14YT0RF2KWVBXNPLBQHK0A5GC5LIBYNXI0' # your Foursquare ID
CLIENT_SECRET = 'QK340CGLIQMUKOFHWPEQXUP0CUITWJY05OR4RGKALKBWS4FS' # your Foursquare Secret
VERSION = '20200622' # Foursquare API version

print('sahildesai360@gmail.com:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

sahildesai360@gmail.com:
CLIENT_ID: KTN2YDVCZLG44W14YT0RF2KWVBXNPLBQHK0A5GC5LIBYNXI0
CLIENT_SECRET:QK340CGLIQMUKOFHWPEQXUP0CUITWJY05OR4RGKALKBWS4FS


In [46]:
radius = 10000
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [47]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3400, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,Chatime Willowdale,43.791326,-79.367506,Bubble Tea Shop
1,M2H,North York,Hillcrest Village,43.803762,-79.363452,Courtyard by Marriott Toronto Northeast/Markham,43.816659,-79.348556,Hotel
2,M2H,North York,Hillcrest Village,43.803762,-79.363452,Shiso Tree Cafe,43.815438,-79.344588,Japanese Restaurant
3,M2H,North York,Hillcrest Village,43.803762,-79.363452,Congee Wong 天皇名粥,43.790579,-79.365815,Chinese Restaurant
4,M2H,North York,Hillcrest Village,43.803762,-79.363452,J-Town,43.815542,-79.344459,Shopping Plaza


In [48]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M2H,North York,Hillcrest Village,100,100,100,100,100,100
M2J,North York,"Fairview, Henry Farm, Oriole",100,100,100,100,100,100
M2K,North York,Bayview Village,100,100,100,100,100,100
M2L,North York,"York Mills, Silver Hills",100,100,100,100,100,100
M2M,North York,"Willowdale, Newtonbrook",100,100,100,100,100,100
M2N,North York,"Willowdale, Willowdale East",100,100,100,100,100,100
M2P,North York,York Mills West,100,100,100,100,100,100
M2R,North York,"Willowdale, Willowdale West",100,100,100,100,100,100
M3A,North York,Parkwoods,100,100,100,100,100,100


48 unique venues

In [50]:
#unique venue categories
print(len(venues_df['VenueCategory'].unique()))

163


In [51]:
york_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
york_onehot['PostalCode'] = venues_df['PostalCode'] 
york_onehot['Borough'] = venues_df['Borough'] 
york_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(york_onehot.columns[-3:]) + list(york_onehot.columns[:-3])
york_onehot = york_onehot[fixed_columns]

print(york_onehot.shape)
york_onehot.head()

(3400, 166)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,M2H,North York,Hillcrest Village,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M2H,North York,Hillcrest Village,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M2H,North York,Hillcrest Village,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M2H,North York,Hillcrest Village,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M2H,North York,Hillcrest Village,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
york_grouped = york_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(york_grouped.shape)
york_grouped

(34, 166)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,M2H,North York,Hillcrest Village,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00
1,M2J,North York,"Fairview, Henry Farm, Oriole",0.00,0.00,0.00,0.01,0.00,0.01,0.01,...,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00
2,M2K,North York,Bayview Village,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00
3,M2L,North York,"York Mills, Silver Hills",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00
4,M2M,North York,"Willowdale, Newtonbrook",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00
5,M2N,North York,"Willowdale, Willowdale East",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00
6,M2P,North York,York Mills West,0.00,0.01,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00
7,M2R,North York,"Willowdale, Willowdale West",0.00,0.01,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00
8,M3A,North York,Parkwoods,0.00,0.00,0.00,0.01,0.00,0.01,0.01,...,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.01,0.00,0.00
9,M3B,North York,Don Mills,0.00,0.00,0.00,0.01,0.00,0.00,0.01,...,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00


# top 7 venues for each PostalCode

In [53]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = york_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = york_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = york_grouped['Neighborhoods']

for ind in np.arange(york_grouped.shape[0]):
    row_categories = york_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(34, 10)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M2H,North York,Hillcrest Village,Chinese Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Caribbean Restaurant,Bakery,Hotel,Supermarket
1,M2J,North York,"Fairview, Henry Farm, Oriole",Caribbean Restaurant,Supermarket,Middle Eastern Restaurant,Bakery,Chinese Restaurant,Japanese Restaurant,Grocery Store
2,M2K,North York,Bayview Village,Middle Eastern Restaurant,Japanese Restaurant,Bakery,Supermarket,Bagel Shop,Coffee Shop,Park
3,M2L,North York,"York Mills, Silver Hills",Café,Bakery,Middle Eastern Restaurant,Park,Coffee Shop,Caribbean Restaurant,Grocery Store
4,M2M,North York,"Willowdale, Newtonbrook",Bakery,Supermarket,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Bagel Shop,Park
5,M2N,North York,"Willowdale, Willowdale East",Coffee Shop,Middle Eastern Restaurant,Supermarket,Park,Café,Bagel Shop,Grocery Store
6,M2P,North York,York Mills West,Café,Coffee Shop,Park,Bakery,Middle Eastern Restaurant,Italian Restaurant,Liquor Store
7,M2R,North York,"Willowdale, Willowdale West",Coffee Shop,Supermarket,Bagel Shop,Park,Liquor Store,Thai Restaurant,Bakery
8,M3A,North York,Parkwoods,Café,Bakery,Caribbean Restaurant,Coffee Shop,Park,Grocery Store,Supermarket
9,M3B,North York,Don Mills,Park,Café,Bakery,Grocery Store,Coffee Shop,Liquor Store,Italian Restaurant


## clustering

In [55]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

york_grouped_clustering = york_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 1, 4, 4, 1, 4, 1, 1])

In [58]:
york_merged = toronto_df_new.copy()


york_merged["Cluster Labels"] = kmeans.labels_


york_merged = york_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(york_merged.shape)
york_merged.head() 

(34, 13)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4,Chinese Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Caribbean Restaurant,Bakery,Hotel,Supermarket
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4,Caribbean Restaurant,Supermarket,Middle Eastern Restaurant,Bakery,Chinese Restaurant,Japanese Restaurant,Grocery Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,4,Middle Eastern Restaurant,Japanese Restaurant,Bakery,Supermarket,Bagel Shop,Coffee Shop,Park
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,1,Café,Bakery,Middle Eastern Restaurant,Park,Coffee Shop,Caribbean Restaurant,Grocery Store
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,4,Bakery,Supermarket,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Bagel Shop,Park


In [59]:
# sorting results by cluster labels
print(york_merged.shape)
york_merged.sort_values(["Cluster Labels"], inplace=True)
york_merged

(34, 13)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
16,M3N,North York,Downsview,43.761631,-79.520999,0,Italian Restaurant,Grocery Store,Hotel,Pizza Place,Coffee Shop,Furniture / Home Store,Liquor Store
31,M9L,North York,Humber Summit,43.756303,-79.565963,0,Hotel,Italian Restaurant,Sporting Goods Shop,Pizza Place,Coffee Shop,Bakery,Liquor Store
32,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,0,Hotel,Grocery Store,Coffee Shop,Italian Restaurant,Chinese Restaurant,Liquor Store,Indian Restaurant
14,M3L,North York,Downsview,43.739015,-79.506944,0,Café,Liquor Store,Coffee Shop,Italian Restaurant,Grocery Store,Bakery,Asian Restaurant
12,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0,Coffee Shop,Thai Restaurant,Liquor Store,Steakhouse,Middle Eastern Restaurant,Supermarket,Grocery Store
23,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,1,Park,Café,Coffee Shop,Bakery,Italian Restaurant,Liquor Store,Grocery Store
15,M3M,North York,Downsview,43.728496,-79.495697,1,Café,Italian Restaurant,Coffee Shop,Park,Bar,Bakery,Liquor Store
13,M3K,North York,Downsview,43.737473,-79.464763,1,Park,Grocery Store,Middle Eastern Restaurant,Coffee Shop,Supermarket,Café,Bar
11,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,1,Coffee Shop,Café,Park,Bakery,Grocery Store,Middle Eastern Restaurant,Bagel Shop
33,M9N,York,Weston,43.706876,-79.518188,1,Café,Italian Restaurant,Park,Bar,Bakery,Brewery,Coffee Shop


In [62]:
from matplotlib import cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['PostalCode'], york_merged['Borough'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

## cluster analysis

In [63]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
16,North York,0,Italian Restaurant,Grocery Store,Hotel,Pizza Place,Coffee Shop,Furniture / Home Store,Liquor Store
31,North York,0,Hotel,Italian Restaurant,Sporting Goods Shop,Pizza Place,Coffee Shop,Bakery,Liquor Store
32,North York,0,Hotel,Grocery Store,Coffee Shop,Italian Restaurant,Chinese Restaurant,Liquor Store,Indian Restaurant
14,North York,0,Café,Liquor Store,Coffee Shop,Italian Restaurant,Grocery Store,Bakery,Asian Restaurant
12,North York,0,Coffee Shop,Thai Restaurant,Liquor Store,Steakhouse,Middle Eastern Restaurant,Supermarket,Grocery Store


In [64]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
23,North York,1,Park,Café,Coffee Shop,Bakery,Italian Restaurant,Liquor Store,Grocery Store
15,North York,1,Café,Italian Restaurant,Coffee Shop,Park,Bar,Bakery,Liquor Store
13,North York,1,Park,Grocery Store,Middle Eastern Restaurant,Coffee Shop,Supermarket,Café,Bar
11,North York,1,Coffee Shop,Café,Park,Bakery,Grocery Store,Middle Eastern Restaurant,Bagel Shop
33,York,1,Café,Italian Restaurant,Park,Bar,Bakery,Brewery,Coffee Shop
8,North York,1,Café,Bakery,Caribbean Restaurant,Coffee Shop,Park,Grocery Store,Supermarket
6,North York,1,Café,Coffee Shop,Park,Bakery,Middle Eastern Restaurant,Italian Restaurant,Liquor Store
9,North York,1,Park,Café,Bakery,Grocery Store,Coffee Shop,Liquor Store,Italian Restaurant
3,North York,1,Café,Bakery,Middle Eastern Restaurant,Park,Coffee Shop,Caribbean Restaurant,Grocery Store


In [65]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
10,North York,2,Café,Park,Coffee Shop,Gastropub,Italian Restaurant,Brewery,Bakery
22,East York,2,Park,Coffee Shop,Beach,Café,Historic Site,Gastropub,Bakery
20,East York,2,Park,Café,Coffee Shop,Italian Restaurant,Brewery,Gastropub,Bakery
19,East York,2,Park,Coffee Shop,Beach,Café,Grocery Store,Farmers Market,Brewery
21,East York,2,Park,Coffee Shop,Café,Gastropub,Italian Restaurant,Historic Site,Grocery Store
17,North York,2,Park,Café,Brewery,Beach,Grocery Store,Italian Restaurant,Middle Eastern Restaurant
18,East York,2,Park,Café,Beach,Coffee Shop,Gastropub,Brewery,Grocery Store


In [66]:
york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
24,North York,3,Park,Café,Grocery Store,Bookstore,Bakery,Bar,Dog Run
25,North York,3,Park,Bakery,Sandwich Place,Café,Grocery Store,Coffee Shop,Bar
26,York,3,Park,Bakery,Café,Coffee Shop,Sandwich Place,Grocery Store,Bookstore
27,York,3,Park,Bakery,Sandwich Place,Coffee Shop,Café,Bookstore,Bar
28,North York,3,Park,Grocery Store,Bakery,Café,Bookstore,Ice Cream Shop,Coffee Shop
29,York,3,Park,Bakery,Coffee Shop,Sandwich Place,Café,Ice Cream Shop,Grocery Store
30,York,3,Park,Bakery,Café,Ice Cream Shop,Coffee Shop,Mexican Restaurant,Brewery


In [67]:
york_merged.loc[york_merged['Cluster Labels'] == 4, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
4,North York,4,Bakery,Supermarket,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Bagel Shop,Park
5,North York,4,Coffee Shop,Middle Eastern Restaurant,Supermarket,Park,Café,Bagel Shop,Grocery Store
7,North York,4,Coffee Shop,Supermarket,Bagel Shop,Park,Liquor Store,Thai Restaurant,Bakery
2,North York,4,Middle Eastern Restaurant,Japanese Restaurant,Bakery,Supermarket,Bagel Shop,Coffee Shop,Park
1,North York,4,Caribbean Restaurant,Supermarket,Middle Eastern Restaurant,Bakery,Chinese Restaurant,Japanese Restaurant,Grocery Store
0,North York,4,Chinese Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Caribbean Restaurant,Bakery,Hotel,Supermarket


#### Clusters 1,2, and 5 mainly belong to North York. East York takes a dominant portion of cluster 3, but doesnt appear in any of the other clusters. Clusters 1 and 2 are more varied with their venues. Clusters 3 and 4 are mainly park venues. Cluster 5 venues are mainly eating establishments.